<h2>--- Day 6: Probably a Fire Hazard ---</h2>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/oddrationale/AdventOfCode2015FSharp/master?urlpath=lab%2Ftree%2FDay06.ipynb)

<p>Because your neighbors keep defeating you in the holiday house decorating contest year after year, you've decided to deploy one million lights in a <span title="Hey, be glad I'm not asking for the resistance between two points!">1000x1000 grid</span>.</p>
<p>Furthermore, because you've been especially nice this year, Santa has mailed you instructions on how to display the ideal lighting configuration.</p>
<p>Lights in your grid are numbered from 0 to 999 in each direction; the lights at each corner are at <code>0,0</code>, <code>0,999</code>, <code>999,999</code>, and <code>999,0</code>. The instructions include whether to <code>turn on</code>, <code>turn off</code>, or <code>toggle</code> various inclusive ranges given as coordinate pairs.  Each coordinate pair represents opposite corners of a rectangle, inclusive; a coordinate pair like <code>0,0 through 2,2</code> therefore refers to 9 lights in a 3x3 square.  The lights all start turned off.</code>
<p>To defeat your neighbors this year, all you have to do is set up your lights by doing the instructions Santa sent you in order.</p>
<p>For example:</p>
<ul>
<li><code>turn on 0,0 through 999,999</code> would turn on (or leave on) every light.</li>
<li><code>toggle 0,0 through 999,0</code> would toggle the first line of 1000 lights, turning off the ones that were on, and turning on the ones that were off.</li>
<li><code>turn off 499,499 through 500,500</code> would turn off (or leave off) the middle four lights.</li>
</ul>
<p>After following the instructions, <em>how many lights are lit</em>?</p>

In [ ]:
let input = File.ReadAllLines @"input/06.txt"

In [ ]:
open System.Text.RegularExpressions

In [ ]:
type State = Off | On

In [ ]:
type Action = Toggle | Turn of State

In [ ]:
type Point = { X: int; Y: int }

In [ ]:
type Instruction = 
    { Action: Action
      First: Point
      Last: Point }

In [ ]:
let parseLine (s: string) = 
    let re = Regex.Match(s, @"(toggle|turn off|turn on) (\d+,\d+) through (\d+,\d+)")
    let first = re.Groups.[2].Value.Split(',')
    let last = re.Groups.[3].Value.Split(',')
    { Action = 
        match re.Groups.[1].Value with
        | "toggle" -> Toggle
        | "turn off" -> Turn Off
        | "turn on" -> Turn On
        | _ -> raise <| FormatException("Invalid format.")
      First = { X = int first.[0]; Y = int first.[1] }
      Last = { X = int last.[0]; Y = int last.[1] } }

In [ ]:
let turn (state: State) (first: Point) (last: Point) (grid: State[][]) =
    for i in first.X..last.X do
        for j in first.Y..last.Y do
            grid.[i].[j] <- state

In [ ]:
let toggle (first: Point) (last: Point) (grid: State[][]) = 
    for i in first.X..last.X do
        for j in first.Y..last.Y do
            match grid.[i].[j] with
            | Off -> grid.[i].[j] <- On
            | On -> grid.[i].[j] <- Off

In [ ]:
let grid = [| for x in 0..999 do [| for row in 0..999 do Off |] |]
input
|> Seq.map parseLine
|> Seq.iter (fun i -> 
    match i.Action with 
    | Toggle -> toggle i.First i.Last grid
    | Turn Off -> turn Off i.First i.Last grid
    | Turn On -> turn On i.First i.Last grid)

In [ ]:
grid
|> Seq.collect (fun row -> row)
|> Seq.filter (fun state -> state = On)
|> Seq.length

543903

<h2 id="part2">--- Part Two ---</h2>

<p>You just finish implementing your winning light pattern when you realize you mistranslated Santa's message from Ancient Nordic Elvish.</p>
<p>The light grid you bought actually has individual brightness controls; each light can have a brightness of zero or more.  The lights all start at zero.</p>
<p>The phrase <code>turn on</code> actually means that you should increase the brightness of those lights by <code>1</code>.</p>
<p>The phrase <code>turn off</code> actually means that you should decrease the brightness of those lights by <code>1</code>, to a minimum of zero.</p>
<p>The phrase <code>toggle</code> actually means that you should increase the brightness of those lights by <code>2</code>.</p>
<p>What is the <em>total brightness</em> of all lights combined after following Santa's instructions?</p>
<p>For example:</p>
<ul>
<li><code>turn on 0,0 through 0,0</code> would increase the total brightness by <code>1</code>.</li>
<li><code>toggle 0,0 through 999,999</code> would increase the total brightness by <code>2000000</code>.</li>
</ul>

In [ ]:
let turn' (state: State) (first: Point) (last: Point) (grid: int[][]) =
    for i in first.X..last.X do
        for j in first.Y..last.Y do
            match state with
            | On -> grid.[i].[j] <- grid.[i].[j] + 1
            | Off -> 
                match grid.[i].[j] with
                | 0 -> ()
                | _ -> grid.[i].[j] <- grid.[i].[j] - 1

In [ ]:
let toggle' (first: Point) (last: Point) (grid: int[][]) = 
    for i in first.X..last.X do
        for j in first.Y..last.Y do
            grid.[i].[j] <- grid.[i].[j] + 2

In [ ]:
let grid' = [| for x in 0..999 do [| for row in 0..999 do 0 |] |]
input
|> Seq.map parseLine
|> Seq.iter (fun i -> 
    match i.Action with 
    | Toggle -> toggle' i.First i.Last grid'
    | Turn Off -> turn' Off i.First i.Last grid'
    | Turn On -> turn' On i.First i.Last grid')

In [ ]:
grid'
|> Seq.collect (fun row -> row)
|> Seq.sum

14687245

[Prev](Day05.ipynb) | [Next](Day07.ipynb)